In [2]:
# Distribution ✨ jit ❇ Demo 💪 
%load_ext autoreload
%autoreload 2
%env "WANDB_NOTEBOOK_NAME" "hwat-notebook"

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
# from jax.config import config
# config.update('jax_disable_jit', True)
from functools import partial
import numpy as np
import re as regex
from pprint import pprint

import jax
from jax import pmap, grad
from jax import numpy as jnp
from flax.training.train_state import TrainState
from flax.core.frozen_dict import FrozenDict	

from pyfig import Pyfig
from hwat import FermiNet, sample, compute_ke_b, PotentialEnergy
from hwat import PotentialEnergy, sample, compute_ke_b
import wandb

env: "WANDB_NOTEBOOK_NAME"="hwat-notebook"


In [3]:
def collect_stats(k, v, new_d, p='tr', suf='', sep='/', sep_long='-'):
	depth = p.count('/')
	if depth > 1:
		sep = sep_long
	if isinstance(v, dict):
		for k_sub,v_sub in v.items():
			collect_stats(k, v_sub, new_d, p=(p+sep+k_sub))
	else:
		new_d[p+sep+k+suf] = v
	return new_d

def compute_metrix(d:dict, mode='tr'):
	pattern_ignore = [
		'.*Dense_[1-9].*'
		'.*rng_.*',
	]
	fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
	)
	_d = {}
	for k,v in d.items():
		k = fancy.get(k, k)
		if isinstance(v, FrozenDict):
			v = v.unfreeze()

		v = jax.tree_map(lambda x: np.array(jax.device_get(x)), v)
		v_mean = jax.tree_map(lambda x: x.mean() if not np.isscalar(x) else x, v)
		v_std = jax.tree_map(lambda x: x.std(), v)

		_d = collect_stats(k, v_mean, _d, p=mode, suf=r'_\mu$')
		_d = collect_stats(k, v_std, _d, p=mode, suf=r'_\sigma$')

	# return {k:v for k,v in _d.items() if not any([regex.match(k, pat) for pat in pattern_ignore])}
	return {k:v for k,v in _d.items() if not any([pat in k for pat in pattern_ignore])}


In [18]:
c.d

{'project_root': PosixPath('/home/amawi/projects'),
 'project': 'hwat',
 'data_dir': PosixPath('/home/amawi/projects/data'),
 'run_path': PosixPath('/home/amawi/projects/hwat/run.py'),
 'exp_name': 'demo',
 'exp_id': '2SbUE9h',
 'dtype': 'f32',
 'n_step': 20,
 'log_metric_step': 5,
 'log_sample_step': 5,
 'log_state_step': 10,
 'seed': 808017424,
 'rng_init': DeviceArray([[3855292491, 2129030713],
              [ 880713437, 1121735675]], dtype=uint32),
 'data': {'n_b': 16,
  'n_e': 6,
  'n_u': 3,
  'n_d': 3,
  'a': DeviceArray([[0., 0., 0.]], dtype=float32),
  'a_z': DeviceArray([6.], dtype=float32),
  'init_walker': functools.partial(<function init_walker at 0x7f6975c549d0>, n_b=16, n_u=3, n_d=3, center=DeviceArray([[0., 0., 0.]], dtype=float32), std=0.1),
  'corr_len': 20,
  'acc_target': 0.5},
 'model': {'n_sv': 32,
  'n_pv': 16,
  'n_fb': 16,
  'n_det': 1,
  'n_fb_out': 128,
  'terms_s_emb': ['x_rlen'],
  'terms_p_emb': ['xx'],
  'compute_s_emb': functools.partial(<function compute

In [36]:
# vars(state)
jax.devices(state.params['params'])

TypeError: unhashable type: 'ShardedDeviceArray'

In [44]:
from utils import flat_any
# import time
# from multiprocess import Manager, Process, Value

# manager = Manager()
# list1 = manager.list(["a"])
# go = Value('b', True)

# def worker(list1, go):
#     while go.value:
#         time.sleep(1)
#         print(list1)
#     return

# p = Process(target=worker, args=(list1, go))
# p.start()

debug_args = {'n_e': 6,'n_u': 3,'n_b': 4, 'n_sv': 8, 'n_pv': 8, 'corr_len': 2, 'n_step': 20,'log_metric_step': 5,'exp_name':'debug'}
class arg_cls(Pyfig):
    def __init__(_i):
        pass
arg_cls.data.n_e = 6
arg_cls.data.n_u = 6
arg_cls.data.n_b = 6
arg_cls.n_step = 20
arg_cls.log_metric_step = 5
arg_cls.exp_name = 'demo'

args = flat_any(arg_cls().d)

c = Pyfig(wandb_mode='online', args=debug_args, get_sys_arg=False)
print('Pyfig ✅')

part_sample = partial(sample, acc_target=c.data.acc_target)

@partial(pmap, in_axes=(0, 0, 0, 0))
def train_step(rng, state, x, deltar):
    x, v_sam = part_sample(rng, state, x, deltar)

    pe = partial(PotentialEnergy(a=c.data.a, a_z=c.data.a_z).apply, {})(x)
    ke = compute_ke_b(state, x)
    e = pe + ke

    def loss(_params):
        return (e * state.apply_fn(_params, x)).sum()
    
    grads = grad(loss)(state.params)

    v = dict(
        grads=grads,
        pe=pe,
        ke=ke,
        e=e,
        deltar=v_sam['deltar'],
        rng=rng,
        x=x,
        acc=v_sam['acc']
    )
    state = state.apply_gradients(grads=grads)
    return state, v
print('Train Step ✅')

rng = c.rng_init
x = c.data.init_walker(rng, n_b=c.data.n_b)
deltar = jnp.ones((c.n_device, 1), dtype=x.dtype)*0.02
print(f'Init: x {x.shape} rng {rng.shape} ✅')

@partial(jax.pmap, axis_name='b', in_axes=(0,0))
def create_train_state(rng, x):
    model = c.partial(FermiNet)  
    params = model.init(rng, x)
    state = TrainState.create(apply_fn=model.apply, params=params, tx=c.opt.tx)
    return state

state = create_train_state(rng, x)
print('Model ✅')

@partial(jax.pmap, axis_name='b', in_axes=(0,0,0,0))
def equil(rng, state, x, deltar):
    for _ in range(10):
        x, v_sam = part_sample(rng, state, x, deltar)
        rng, deltar = v_sam['rng'], v_sam['deltar']
    return x, v_sam
print('Equil ✅')

wandb.define_metric("*", step_metric="eq/step")
for step in range(1, 501):
    x, v_sam = equil(rng, state, x, deltar)
    rng, deltar, acc = v_sam['rng'], v_sam['deltar'], v_sam['acc']
    if not (step % c.log_metric_step):
        wandb.log({'eq/step':step, **v_sam})
print('Walkers ✅ Training Variables ✅')

@jax.pmap
def update_model(state, grads):
  return state.apply_gradients(grads=grads)
print('Update ✅')

wandb.define_metric("*", step_metric="tr/step")
for step in range(1, c.n_step+1):
    state, data = train_step(rng, state, x, deltar)
    rng, deltar, x, grads = data['rng'], data['deltar'], data['x'], data['grads']

    if not (step % c.log_metric_step):
        metrix = compute_metrix(data)
        wandb.log({'tr/step':step, **metrix})
        m = ' '.join([f'{k} {v:.5f} ' for k,v in metrix.items() if 'E_' in k])

    if not (step % c.log_state_step):
        ...

print('Go seek: ', c.wandb_c.wandb_run_path)
wandb.finish()

setting:  n_e 6
setting:  n_u 3
setting:  n_b 4
setting:  n_sv 8
setting:  n_pv 8
setting:  corr_len 2
setting:  n_step 20
setting:  log_metric_step 5
setting:  exp_name debug
0 args unmerged: ✅
Path:  /home/amawi/projects/hwat/exp/debug/HMJuihb ✅


run:  xmax1/hwat/1gl2w1yz ✅
Pyfig ✅
Train Step ✅
Init: x (2, 4, 6, 3) rng (2, 2) ✅
Model ✅
Equil ✅


2022-11-30 11:08:38.101708: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module pmap_equil] Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2022-11-30 11:14:29.475447: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 7m51.372586518s

********************************
[Compiling module pmap_equil] Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


Walkers ✅ Training Variables ✅
Update ✅


AttributeError: 'Pyfig' object has no attribute 'wandb'

In [ ]:
# https://ipyparallel.readthedocs.io/en/latest/

%wandb c.wandb_c.wandb_run_path

In [ ]:
# 💓 
class tr_data:
    x:jnp.ndarray=x
    rng:jnp.ndarray=rng